In [59]:
from pprint import pprint

# Build a 10-period binomial short rate lattice
# Parameters
n = 10                  # Number of periods
r_0 = 0.05              # Initial short rate (5%)
u = 1.1                 # Up factor
d = 0.9                 # Down factor
q = 0.5                 # Risk-neutral probability

# Build short-rate lattice
short_rate_lattice = [[0]*(n + 1) for _ in range(n + 1)]
short_rate_lattice[0][0] = r_0

for j in range(1, n + 1):
    for i in range(0, j + 1):
        if i == 0:
            short_rate_lattice[i][j] = short_rate_lattice[i][j - 1] * u
        else:
            short_rate_lattice[i][j] = short_rate_lattice[i - 1][j - 1] * d

print('\n'.join('  '.join(f"{x:7.4f}" if isinstance(x, float) else "       " for x in row) for row in short_rate_lattice))   

zcb_price_lattice = [[0]*(n + 1) for _ in range(n + 1)]
for i in range(n + 1):
    zcb_price_lattice[i][n] = 100.0

# Backward induction to calculate ZCB prices
for j in range(n - 1, -1, -1):
    for i in range(0, j + 1):
        zcb_price_lattice[i][j] = (q * zcb_price_lattice[i][j + 1] + (1 - q) * zcb_price_lattice[i + 1][j + 1]) / (1 + short_rate_lattice[i][j])

# Print ZCB price lattice
print("\nZero-Coupon Bond Price Lattice:")
print('\n'.join('  '.join(f"{x:7.4f}" if isinstance(x, float) else "       " for x in row) for row in zcb_price_lattice))    

zcb_4_year_lattice = [[0]* 5 for _ in range(5)]
for i in range(5):
    zcb_4_year_lattice[i][4] = 100.0

for j in range(3, -1, -1):
    for i in range(0, j + 1):
        zcb_4_year_lattice[i][j] = (q * zcb_4_year_lattice[i][j + 1] + (1 - q) * zcb_4_year_lattice[i + 1][j + 1]) / (1 + short_rate_lattice[i][j])

print("\nZero-Coupon Bond Price Lattice:")
print('\n'.join('  '.join(f"{x:7.4f}" if isinstance(x, float) else "       " for x in row) for row in zcb_4_year_lattice))    

forward_price = zcb_price_lattice[0][0] / zcb_4_year_lattice[0][0] * 100

print(f"\nForward Price for 4-Year Zero-Coupon Bond: {forward_price:.4f}")

future_lattice = [[0]*5 for _ in range(5)]
for i in range(5):
    future_lattice[i][4] = zcb_price_lattice[i][4]

for j in range(3, -1, -1):
    for i in range(0, j + 1):
        future_lattice[i][j] = (q * future_lattice[i][j + 1] + (1 - q) * future_lattice[i + 1][j + 1]) 


print('\n'.join('  '.join(f"{x:7.4f}" if isinstance(x, float) else "       " for x in row) for row in future_lattice))    

option_expiration = 6  # 6 years
option_strike = 80.0  # Strike price of the option
lattice_size = option_expiration+1  # Size of the lattice (0 to 6)

american_option_lattice = [[0.0] * lattice_size for _ in range(lattice_size)]
for i in range(lattice_size):
    american_option_lattice[i][lattice_size - 1] = max(0.0, zcb_price_lattice[i][lattice_size - 1] - option_strike)

for j in range(lattice_size - 2, -1, -1):
    for i in range(j + 1):
        exercise_value = max(0.0, zcb_price_lattice[i][j] - option_strike)
        continuation_value = (q * american_option_lattice[i][j + 1] + (1 - q) * american_option_lattice[i + 1][j + 1]) / (1 + short_rate_lattice[i][j])
        american_option_lattice[i][j] = max(exercise_value, continuation_value)

print("\nAmerican Option Price Lattice:")
print('\n'.join('  '.join(f"{x:7.4f}" if isinstance(x, float) else "       " for x in row) for row in american_option_lattice))

 0.0500   0.0550   0.0605   0.0666   0.0732   0.0805   0.0886   0.0974   0.1072   0.1179   0.1297
          0.0450   0.0495   0.0545   0.0599   0.0659   0.0725   0.0797   0.0877   0.0965   0.1061
                   0.0405   0.0446   0.0490   0.0539   0.0593   0.0652   0.0717   0.0789   0.0868
                            0.0365   0.0401   0.0441   0.0485   0.0534   0.0587   0.0646   0.0710
                                     0.0328   0.0361   0.0397   0.0437   0.0480   0.0528   0.0581
                                              0.0295   0.0325   0.0357   0.0393   0.0432   0.0475
                                                       0.0266   0.0292   0.0322   0.0354   0.0389
                                                                0.0239   0.0263   0.0289   0.0318
                                                                         0.0215   0.0237   0.0260
                                                                                  0.0194   0.0213
                    